# Playground

A safe space for experimentation.

In [2]:
%load_ext autoreload
%autoreload 2

import os
import schedule
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from etl_db_manager import ETLDataBaseManager
from etl_task import ETLTask, ETLTaskFunction
from utility import base64encode_obj, base64decode_obj

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [3]:
def fetch_data_airnoise(sensor_id:str, time_start:datetime, time_end:datetime):
    """ Fetches air/noise data from the API
        at: https://data.smartdublin.ie/sonitus-api. 
        @param sensor_id: Serial number of the sensor.
        @param time_start: Time range starting point.
        @param time_end: Time range ending point.
        @return: Data from API or [].
    """
    data = []
    try:
        res = requests.post(f"https://data.smartdublin.ie/sonitus-api/api/data", json={ 
            'username': "dublincityapi",
            'password': "Xpa5vAQ9ki",
            'monitor': sensor_id,
            'start': time_start.timestamp(),
            'end': time_end.timestamp()
        })
        print("[DEBUG] res =", res)
        data = res.json()
    except Exception as e:
          raise print(f'Failed to fetch air noise data from source: {e}')
    return data

In [4]:
def load_data_noise():
    """ Loads last 30 mins worth of noise data 
        from multiple noise sensors.
    """
    dt_now = datetime.now() # Time now.
    dt_past = dt_now - timedelta(minutes=30) # Time 30 mins ago.
    # Fetch data from multiple sensors.
    data_sensors = {"01749":[], "01508":[], "10118":[], "01548":[], "10115":[]}
    for sensor in data_sensors.keys():
        data_sensors[sensor] = fetch_data_airnoise(
            sensor_id=sensor, 
            time_start=dt_past, 
            time_end=dt_now
        )
    return data_sensors

## Schedule Load Save

In [43]:
def load_data_bikes():
    """ Loads last 30 min snapshot of dublin bike stands. """
    import requests
    data = []
    try:
        res = requests.get(f"http://localhost:8000/bikes/snapshot/")
        data = res.json()['data']
        print(res.json()['message'])
    except Exception as e:
          data = []
          print(f'Failed to fetch dublin bikes data from source: {e}')
          raise Exception(f'Failed to fetch dublin bikes data from source: {e}')
    return data

def transform_data_bikes(data):
    """
    Transforms bikes data to be in a desireable format for saving.
    @param data: Data to be transformed.
    """
    import pytz
    import pandas as pd
    from datetime import datetime

    df = pd.DataFrame(data)
    df['usage_percent'] = df['available_bikes']/df['bike_stands']
    df['usage_percent'] = df['usage_percent'].round(2)
    df['status'] = df['status'].str.lower()
    df = df[[
        'station_id', 'bike_stands', 
        'available_bikes', 'usage_percent', 
        'last_update', 'status'
    ]]
    print("Bike data transformed. Usage % computed and last update made lowercase.")

    return df.to_dict(orient='records')

def save_data_bikes(data):
    """ 
    Saves given data to csv file. 
    @param data: Data to be saved.
    """
    import requests
    try:
        res = requests.post(
            url="http://localhost:8000/bikes/snapshot/", 
            json={'snapshot':data}
        )
        print(res.json()['message'])
    except Exception as e:
        print(f'Failed to save bikes data. {e}')
        raise Exception(f'Failed to save bikes data. {e}')

In [42]:
data = load_data_bikes()
data = transform_data_bikes(data)
save_data_bikes(data)

Success. Bike stations last 30 mins snapshot fetched from DCC API.
Bike data transformed. Usage % computed and last update made lowercase.
Success. Saved snapshot of bike data for every station.


In [39]:
data

[{'station_id': 1,
  'bike_stands': 31,
  'available_bikes': 0,
  'usage_percent': 0.0,
  'last_update': '2024-03-19T22:37:48',
  'status': 'open'},
 {'station_id': 2,
  'bike_stands': 20,
  'available_bikes': 1,
  'usage_percent': 0.05,
  'last_update': '2024-03-19T22:33:50',
  'status': 'open'},
 {'station_id': 3,
  'bike_stands': 20,
  'available_bikes': 3,
  'usage_percent': 0.15,
  'last_update': '2024-03-19T22:38:43',
  'status': 'open'},
 {'station_id': 4,
  'bike_stands': 20,
  'available_bikes': 15,
  'usage_percent': 0.75,
  'last_update': '2024-03-19T22:29:45',
  'status': 'open'},
 {'station_id': 5,
  'bike_stands': 40,
  'available_bikes': 29,
  'usage_percent': 0.725,
  'last_update': '2024-03-19T22:37:40',
  'status': 'open'},
 {'station_id': 6,
  'bike_stands': 20,
  'available_bikes': 4,
  'usage_percent': 0.2,
  'last_update': '2024-03-19T22:38:09',
  'status': 'open'},
 {'station_id': 7,
  'bike_stands': 29,
  'available_bikes': 4,
  'usage_percent': 0.13793103448275

In [44]:
# import pandas as pd
# df = pd.DataFrame(data)
# df['usage_percent'] = df['available_bikes']/df['bike_stands']
# df['status'] = df['status'].str.lower()
# df = df[['station_id', 'bike_stands', 'available_bikes', 'usage_percent', 'last_update', 'status']]
# df.to_dict(orient='records')

In [47]:
# Toy data load and save functions.
def load_data(): 
    print(f'Loaded data.')
    return [1, 2, 3]

def transform_data(data):
    data_transformed = data*10
    print(f'Transformed data {data} into {data_transformed}.')
    return data_transformed

def save_data(data): 
    print(f'Saved data {data}.')

In [9]:
def make_post_request(url, data={}):
    """
    Makes a post request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.post(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_get_request(url, data={}):
    """
    Makes a post request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.get(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_delete_request(url, data={}):
    """
    Makes a delete request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.delete(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_put_request(url, task_name, new_values):
    """
    Makes a put request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.put(
        url=url, 
        params={"task_name": task_name}, 
        json=new_values
    )
    return {'status': response.status_code, 'message': response.json()}

In [45]:
db_man = ETLDataBaseManager(db_name="db_etl", db_path=".")

Database './db_etl.db' available :) .


In [46]:
# Create a new task.
task = ETLTask(
    name='bikes',
    fun_data_load=load_data_bikes,
    fun_data_transform=transform_data_bikes,
    fun_data_save=save_data_bikes,
    repeat_time_unit='seconds',
    repeat_interval=10
)
task_str = base64encode_obj(task)
make_post_request(url="http://127.0.0.1:8003/task", data={"task_str": task_str})

{'status': 200,
 'message': '{"status":200,"message":"Success. Task created and scheduled bikes.","data":[]}'}

In [48]:
# Create a new task.
task = ETLTask(
    name='test',
    fun_data_load=load_data,
    fun_data_transform=transform_data,
    fun_data_save=save_data,
    repeat_time_unit='seconds',
    repeat_interval=3
)
task_str = base64encode_obj(task)
make_post_request(url="http://127.0.0.1:8003/task", data={"task_str": task_str})

{'status': 200,
 'message': '{"status":200,"message":"Success. Task created and scheduled test.","data":[]}'}

In [210]:
# Delete a task.
make_delete_request(url="http://127.0.0.1:8003/task", data={'task_name': 'bikes'})

{'status': 200,
 'message': '{"status":400,"message":"Success. Deleted task bikes.","data":[]}'}

In [178]:
# Start scheduler.
make_get_request(url="http://127.0.0.1:8003/start_scheduler")

{'status': 200,
 'message': '{"status":200,"message":"Scheduler started.","data":[]}'}

In [211]:
# Stop scheduler.
make_get_request(url="http://127.0.0.1:8003/stop_scheduler")

{'status': 200,
 'message': '{"status":200,"message":"Scheduler stopped.","data":[]}'}

In [202]:
make_put_request(
    url="http://127.0.0.1:8003/task", 
    task_name="task1",
    new_values={
        'status': 'scheduled',
        "num_runs": 0
    }
)

{'status': 200,
 'message': {'status': 200,
  'message': "Success. Status of task task1 updated with new values {'status': 'scheduled', 'num_runs': 0}.",
  'data': []}}

In [187]:
# Stop task
make_put_request(
    url="http://127.0.0.1:8003/task/stop", 
    task_name="test",
    new_values={}
)

{'status': 200,
 'message': {'status': 200,
  'message': 'Success. Task test has been stopped.',
  'data': []}}

In [6]:
# Request module import.
response = requests.put(
    url="http://127.0.0.1:8003/import", 
    json=['requests']
)
print(response.json())

{'status': 200, 'message': "Imported module 'requests'. "}
